In [97]:
import pandas as pd
from sqlalchemy import create_engine

Working code to add to SSMS

In [121]:
book_data = pd.read_csv('Data/03_Library Systembook.csv')
len(book_data_raw)

114

In [138]:
import pandas as pd
from sqlalchemy import create_engine

#Define drop count
book_drop_count = 0
customer_drop_count = 0

#Read 03_Library Systembook.csv file
book_data = pd.read_csv('Data/03_Library Systembook.csv')

#Drop rows where empty
na_book_drop = book_data.dropna()
book_drop_count +=  len(book_data) - len(na_book_drop)
book_data = book_data.dropna()

#Drop rows where duplicated --- No duplicates?
dup_book_drop = book_data.drop_duplicates()
book_drop_count +=  len(book_data) - len(dup_book_drop)
book_data = book_data.drop_duplicates()

#Change Book checkout data type to datetime - first remove double " and format d/m/y. Also removes wrong dates e.g. 32/05/2023
book_data['Book checkout'] = book_data['Book checkout'].str.strip('"')

book_data['Book checkout'] = pd.to_datetime(
    book_data['Book checkout'],
    format='%d/%m/%Y',
    errors='coerce'
)

subna_book_drop = book_data.dropna()
book_drop_count += len(book_data) - len(subna_book_drop)
book_data = book_data.dropna(subset=['Book checkout'])

#Change Book Returned data type to datetime and format d/m/y.
book_data['Book Returned'] = pd.to_datetime(book_data['Book Returned'], format='%d/%m/%Y')

#Assuming input error for 2063 year value, change to 2023.
book_data.loc[book_data['Book Returned'].dt.year == 2063, 'Book Returned'] = \
    book_data['Book Returned'] - pd.DateOffset(years=40)

#Assuming date input error where Book checkout > Book Returned - swap rows where affected
bad_dates = book_data['Book Returned'] < book_data['Book checkout']
book_data.loc[bad_dates, ['Book checkout', 'Book Returned']] = book_data.loc[bad_dates, ['Book Returned', 'Book checkout']].values

#Add Loan Duration column
book_data['Loan Duration'] = (book_data['Book Returned'] - book_data['Book checkout']).dt.days

#Read 03_Library SystemCustomers.csv file
customers_data = pd.read_csv('Data/03_Library SystemCustomers.csv')

#Drop rows where empty
na_customer_drop = customers_data.dropna()
customer_drop_count +=  len(customers_data) - len(na_customer_drop)
customers_data = customers_data.dropna()

#Connection details for SQLAlchemy
server = 'localhost'
database = 'LibraryDB'
#username = 'python_app'
#password = 'Password1'
#driver = 'ODBC Driver 17 for SQL Server'

#Create connection string WITHOUT username and password
connection_string = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'

#Create connection string WITH uername and password
#connection_string = (
#    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
#    f"?driver={driver.replace(' ', '+')}"
#)

#Create engine
engine = create_engine(connection_string)

drop_count = {
    'Record Loss: Books': book_drop_count,
    'Record Loss: Customers': customer_drop_count
}

metrics = pd.DataFrame([drop_count])

#Write data to Sql table called books
book_data.to_sql('books', con=engine, if_exists='replace', index=False)
customers_data.to_sql('customers', con=engine, if_exists='replace', index=False)
metrics.to_sql('metrics', con=engine, if_exists='replace', index=False)

print("Data successfully written to LibraryDB.books")
print("Data successfully written to LibraryDB.customers")
print("Data successfully written to LibraryDB.metrics")

Data successfully written to LibraryDB.books
Data successfully written to LibraryDB.customers
Data successfully written to LibraryDB.metrics


In [141]:
import pandas as pd
from sqlalchemy import create_engine

#Define drop count
book_drop_count = 0
customer_drop_count = 0

#Read 03_Library Systembook.csv file
book_data = pd.read_csv('Data/03_Library Systembook.csv')

#Drop rows where empty
na_book_drop = book_data.dropna()
book_drop_count +=  len(book_data) - len(na_book_drop)
book_data = book_data.dropna()

#Drop rows where duplicated --- No duplicates?
dup_book_drop = book_data.drop_duplicates()
book_drop_count +=  len(book_data) - len(dup_book_drop)
book_data = book_data.drop_duplicates()

#Change Book checkout data type to datetime - first remove double " and format d/m/y. Also removes wrong dates e.g. 32/05/2023
book_data['Book checkout'] = book_data['Book checkout'].str.strip('"')

book_data['Book checkout'] = pd.to_datetime(
    book_data['Book checkout'],
    format='%d/%m/%Y',
    errors='coerce'
)

subna_book_drop = book_data.dropna()
book_drop_count += len(book_data) - len(subna_book_drop)
book_data = book_data.dropna(subset=['Book checkout'])

#Change Book Returned data type to datetime and format d/m/y.
book_data['Book Returned'] = pd.to_datetime(book_data['Book Returned'], format='%d/%m/%Y')

#Add Loan Duration column
book_data['Loan Duration'] = (book_data['Book Returned'] - book_data['Book checkout']).dt.days

neg_loans = book_data[book_data['Loan Duration'] < 0]
book_drop_count += len(neg_loans)
book_data = book_data[book_data['Loan Duration'] >= 0]

#Assuming input error for 2063 year value, change to 2023.
#book_data.loc[book_data['Book Returned'].dt.year == 2063, 'Book Returned'] = \
#    book_data['Book Returned'] - pd.DateOffset(years=40)






#Read 03_Library SystemCustomers.csv file
customers_data = pd.read_csv('Data/03_Library SystemCustomers.csv')

#Drop rows where empty
na_customer_drop = customers_data.dropna()
customer_drop_count +=  len(customers_data) - len(na_customer_drop)
customers_data = customers_data.dropna()





#Connection details for SQLAlchemy
server = 'localhost'
database = 'LibraryDB'

#Create connection string WITHOUT username and password
connection_string = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'

#Create engine
engine = create_engine(connection_string)

drop_count = {
    'Record Loss: Books': book_drop_count,
    'Record Loss: Customers': customer_drop_count
}

metrics = pd.DataFrame([drop_count])

#Write data to Sql table called books
book_data.to_sql('books', con=engine, if_exists='replace', index=False)
customers_data.to_sql('customers', con=engine, if_exists='replace', index=False)
metrics.to_sql('metrics', con=engine, if_exists='replace', index=False)

print("Data successfully written to LibraryDB.books")
print("Data successfully written to LibraryDB.customers")
print("Data successfully written to LibraryDB.metrics")

Data successfully written to LibraryDB.books
Data successfully written to LibraryDB.customers
Data successfully written to LibraryDB.metrics


In [ ]:
k Returned data type to datetime and format d/m/y.
book_data['Book Returned'] = pd.to_datetime(book_data['Book Returned'], format='%d/%m/%Y')

#Assuming date input error where Book checkout > Book Returned - swap rows where affected
bad_dates = book_data['Book Returned'] < book_data['Book checkout']
book_data.loc[bad_dates, ['Book checkout', 'Book Returned']] = book_data.loc[bad_dates, ['Book Returned', 'Book checkout']].values

#Read 03_Library SystemCustomers.csv file
customers_data = pd.read_csv('Data/03_Library SystemCustomers.csv')

#Drop rows where empty
na_customer_drop = customers_data.dropna()
customer_drop_count +=  len(customers_data) - len(na_customer_drop)
customers_data = customers_data.dropna()

#Connection details for SQLAlchemy
server = 'localhost'
database = 'LibraryDB'
#username = 'python_app'
#password = 'Password1'
#driver = 'ODBC Driver 17 for SQL Server'

#Create connection string WITHOUT username and password
connection_string = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'

#Create connection string WITH uername and password
#connection_string = (
#    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
#    f"?driver={driver.replace(' ', '+')}"
#)

#Create engine
engine = create_engine(connection_string)

drop_count = {
    'Record Loss: Books': book_drop_count,
    'Record Loss: Customers': customer_drop_count
}

metrics = pd.DataFrame([drop_count])

#Write data to Sql table called books
book_data.to_sql('books', con=engine, if_exists='replace', index=False)
customers_data.to_sql('customers', con=engine, if_exists='replace', index=False)
metrics.to_sql('metrics', con=engine, if_exists='replace', index=False)

print("Data successfully written to LibraryDB.books")
print("Data successfully written to LibraryDB.customers")
print("Data successfully written to LibraryDB.metrics")

In [103]:
import pandas as pd
from sqlalchemy import create_engine

#Define drop count
book_drop_count = 0

#Read 03_Library Systembook.csv file
book_data = pd.read_csv('Data/03_Library Systembook.csv')

#Drop rows where empty
na_book_drop = book_data.dropna()
book_drop_count +=  len(book_data) - len(na_book_drop)
book_data = book_data.dropna()

print(book_drop_count)

94


In [92]:
# Connection details for SQLAlchemy
server = 'localhost'
database = 'LibraryDB'
username = 'python_app'
password = 'Password1'
driver = 'ODBC Driver 17 for SQL Server'

# Create connection string and engine
connection_string = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
#connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver.replace(' ', '+')}"
engine = create_engine(connection_string)

#Write data to Sql table called books
book_data.to_sql('books', con=engine, if_exists='replace', index=False)

print("Data successfully written to LibraryDB.books")


Data successfully written to LibraryDB.books


In [83]:
book_data = pd.read_csv('Data/03_Library Systembook.csv')

In [85]:
book_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 19
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Id                      19 non-null     float64       
 1   Books                   19 non-null     object        
 2   Book checkout           19 non-null     datetime64[ns]
 3   Book Returned           19 non-null     datetime64[ns]
 4   Days allowed to borrow  19 non-null     object        
 5   Customer ID             19 non-null     float64       
dtypes: datetime64[ns](2), float64(2), object(2)
memory usage: 1.0+ KB


In [84]:
#Read csv file
book_data = pd.read_csv('Data/03_Library Systembook.csv')

#Drop rows where empty
book_data = book_data.dropna()

#Change Book checkout data type to datetime - first remove double " and format d/m/y. Also removes wrong dates e.g. 32/05/2023
book_data['Book checkout'] = book_data['Book checkout'].str.strip('"')
book_data['Book checkout'] = pd.to_datetime(
    book_data['Book checkout'],
    format='%d/%m/%Y',
    errors='coerce'
)
book_data = book_data.dropna(subset=['Book checkout'])

#Change Book Returned data type to datetime and format d/m/y.
book_data['Book Returned'] = pd.to_datetime(book_data['Book Returned'], format='%d/%m/%Y')

#Assuming date input error where Book checkout > Book Returned - swap rows where affected
bad_dates = book_data['Book Returned'] < book_data['Book checkout']
book_data.loc[bad_dates, ['Book checkout', 'Book Returned']] = book_data.loc[bad_dates, ['Book Returned', 'Book checkout']].values


In [71]:
book_data.head(20)

,Id,Books,Book checkout,Book Returned,Days allowed to borrow,Customer ID
0,1.0,Catcher in the Rye,2023-02-20,2023-02-25,2 weeks,1.0
1,2.0,Lord of the rings the two towers,2023-03-21,2023-03-24,2 weeks,2.0
2,3.0,Lord of the rings the return of the kind,2023-03-25,2023-03-29,2 weeks,3.0
3,4.0,The hobbit,2023-03-25,2023-04-02,2 weeks,4.0
4,5.0,Dune,2023-03-25,2023-04-02,2 weeks,5.0
5,6.0,Little Women,2023-04-02,2023-05-01,2 weeks,1.0
6,7.0,IT,2023-04-03,2063-04-10,2 weeks,6.0
7,8.0,Misery,2023-04-03,2023-04-15,2 weeks,7.0
8,9.0,Catch 22,2023-04-15,2023-04-16,2 weeks,7.0
9,10.0,Animal Farm,2023-04-20,2023-04-24,2 weeks,2.0


In [58]:
#Read csv file
customers_data = pd.read_csv('Data/03_Library SystemCustomers.csv')

#Drop rows where empty
customers_data = customers_data.dropna()

In [59]:
customers_data.info()
customers_data.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 0 to 8
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Customer ID    8 non-null      float64
 1   Customer Name  8 non-null      object 
dtypes: float64(1), object(1)
memory usage: 192.0+ bytes


,Customer ID,Customer Name
0,1.0,Jane Doe
1,2.0,John Smith
2,3.0,Dan Reeves
4,5.0,William Holden
5,6.0,Jaztyn Forest
6,7.0,Jackie Irving
7,8.0,Matthew Stirling
8,9.0,Emory Ted
